In [3]:
from llama_cpp import Llama
import pandas as pd
import json
import random
from sklearn.metrics import classification_report

In [23]:
df = pd.read_json('data/llm_task1_hetero.json', orient="records")
df.head()

,id,text,label
0,het_857,Those whose child is going through a'phase'may...,1
1,het_368,Bureaucrats cut red tape-- lengthwise.,0
2,het_1204,If they say you look good in glasses that woul...,1
3,het_1043,"Nearing the end of his sentence, the convicted...",1
4,het_1098,A logger went to his doctor because of a probl...,1


In [24]:
test = df
len(test)

445

In [4]:
PATH = 'models/mistral-7b-openorca.Q6_K.gguf'
llm = Llama(model_path=PATH, n_gpu_layers=75, n_ctx=2048)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from models/mistral-7b-openorca.Q6_K.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q6_K     [  4096, 32002,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q6_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q6_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q6_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q6_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q6_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.ffn_down.weight q6_K     [ 14336,  4096,     1,     1 ]

In [5]:
def is_json(strJson):
  try:
    json.loads(strJson)
  except ValueError as e:
    return False
  return True

In [6]:
def generate(prompt, attempt):
  if attempt == 1:
    temp = 0.2
  else:
    temp = round(random.uniform(0.3, 0.6), 1)
  output = llm(prompt, max_tokens=-1, temperature=temp)
  return output['choices'][0]['text']

def parseResult(jsonStr, row):
  if not is_json(jsonStr.strip()):
    return None
  else:
    parsedJson = json.loads(jsonStr.strip())
    return parsedJson

### Task 1: Pun Detection

In [15]:
prompt = f''' ### INSTRUCTION:
You are to classify whether a given sentence is a pun:
Here are examples of the cases you will encounter:
1. An example of a pun - 
The magician got so mad that he pulled his hare out.
{{
  "output": true,
  "explanation": "The pun word 'hare' supports two plausible interpretations of 'hare' meaning a rabbit and 'hair' meaning human hair. In the given sentence, the words 'magician' relates to 'hare' while 'pulled' refers to the second interpretation of 'hair'. Both of these are distinct where one refers to a magician's animal while the other refers to an action done in anger which is pulling your hair."
}}
2. An example of no pun -
I went to the bank.
{{
  "output": false,
  "explanation": "The word 'bank' does have ambiguity here where it supports two plausible interpretations of bank as in a financial institution and bank as in the banks of a river. There are no other words in the sentence that realize both interpretations, hence the sentence is not a pun."
}}
3. An example of no pun -
Let us go home.
{{
  "output": false,
  "explanation": "There is no pun word in the sentence."
}}
Identify whether the input sentence is a pun and explain the result in valid JSON format. Generate a response in the form of a valid JSON object with two keys: output and explanation.\n
### INPUT:\n
'''

In [25]:
results = []
i = 0
for _, row in test.iterrows():
  prepared_prompt = prompt + row['text'] + "\n### OUTPUT:\n"
  json_out = ''
  attempt = 0
  while True:
    attempt += 1
    json_out = generate(prepared_prompt, attempt)
    parsedJson = parseResult(json_out, row)
    print(f"{i} -- {attempt}")
    if parsedJson is not None:
      parsedJson['id'] = row['id']
      parsedJson['text'] = row['text']
      results.append(parsedJson)
      i+=1
      break


Llama.generate: prefix-match hit


0 -- 1



llama_print_timings:        load time =    2260.34 ms
llama_print_timings:      sample time =      12.21 ms /   108 runs   (    0.11 ms per token,  8848.11 tokens per second)
llama_print_timings: prompt eval time =    2870.89 ms /    25 tokens (  114.84 ms per token,     8.71 tokens per second)
llama_print_timings:        eval time =    5558.63 ms /   107 runs   (   51.95 ms per token,    19.25 tokens per second)
llama_print_timings:       total time =    8608.77 ms
Llama.generate: prefix-match hit


1 -- 1



llama_print_timings:        load time =    2260.34 ms
llama_print_timings:      sample time =      14.97 ms /   127 runs   (    0.12 ms per token,  8484.77 tokens per second)
llama_print_timings: prompt eval time =     185.06 ms /    17 tokens (   10.89 ms per token,    91.86 tokens per second)
llama_print_timings:        eval time =    6585.02 ms /   126 runs   (   52.26 ms per token,    19.13 tokens per second)
llama_print_timings:       total time =    6974.11 ms
Llama.generate: prefix-match hit


...


llama_print_timings:        load time =    2260.34 ms
llama_print_timings:      sample time =      10.19 ms /    99 runs   (    0.10 ms per token,  9713.50 tokens per second)
llama_print_timings: prompt eval time =     189.53 ms /    21 tokens (    9.03 ms per token,   110.80 tokens per second)
llama_print_timings:        eval time =    5077.00 ms /    98 runs   (   51.81 ms per token,    19.30 tokens per second)
llama_print_timings:       total time =    5401.34 ms
Llama.generate: prefix-match hit


443 -- 1



llama_print_timings:        load time =    2260.34 ms
llama_print_timings:      sample time =       2.96 ms /    29 runs   (    0.10 ms per token,  9780.78 tokens per second)
llama_print_timings: prompt eval time =     195.34 ms /    28 tokens (    6.98 ms per token,   143.34 tokens per second)
llama_print_timings:        eval time =    1446.34 ms /    28 runs   (   51.65 ms per token,    19.36 tokens per second)
llama_print_timings:       total time =    1680.48 ms
Llama.generate: prefix-match hit


444 -- 1



llama_print_timings:        load time =    2260.34 ms
llama_print_timings:      sample time =       3.23 ms /    29 runs   (    0.11 ms per token,  8989.46 tokens per second)
llama_print_timings: prompt eval time =     183.56 ms /    16 tokens (   11.47 ms per token,    87.16 tokens per second)
llama_print_timings:        eval time =    1446.19 ms /    28 runs   (   51.65 ms per token,    19.36 tokens per second)
llama_print_timings:       total time =    1671.89 ms


In [26]:
results_df = pd.DataFrame(results, columns=['id', 'text', 'output', 'explanation'])
results_df.to_json('outputs-task1-hetero-openocra7b-nohumor.json', orient="records")
len(results_df)

445

In [27]:
results_df['output'] = results_df['output'].replace({True: 1, False: 0})

In [28]:

y_true = [int(x) for x in list(test['label'])]
y_pred = [int(x) for x in list(results_df['output'])]

In [29]:
print(classification_report(y_true, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.3938    0.9055    0.5489       127
           1     0.9216    0.4434    0.5987       318

    accuracy                         0.5753       445
   macro avg     0.6577    0.6745    0.5738       445
weighted avg     0.7710    0.5753    0.5845       445



### Task 2 and 3: Pun Location and Interpretation

In [7]:
def clean_sentence(text):
  res = text.replace(" ' ' ", '" ')
  res = res.replace(" ' ", "'")
  res = res.replace(" , ", ", ")
  res = res.replace(" . ", ". ")
  res = res.replace(" .", ". ")
  res = res.replace(" ! ", "! ")
  res = res.replace(" !", "! ")
  res = res.replace(" ? ", "? ")
  res = res.replace(" ?", "? ")
  res = res.replace(" - ", "-")
  return res.strip()

In [15]:
df = pd.read_json('data/llm_task2_hetero.json', orient="records")
df['sentence'] = df['sentence'].apply(clean_sentence)
df = df.rename(columns={'sentence': 'text'})
df.head()

,text,pun_word
0,In some countries there is a lot of Manuel labor.,Manuel
1,Someone who attempts long distance fraud is kn...,phony
2,"In the delivery room, the unborn infant expect...",berth
3,Beautifully manicured lawns are highly sod after.,sod
4,"I saw something similar to moss the other day,...",lichen


In [16]:
test = df
len(test)

318

In [17]:
prompt = f''' ### INSTRUCTION:
You are to identify the pun word in a given sentence. It is guaranteed that the sentence has a pun word.\n
Also, list all the words that realize each interpretation of the pun word. If there are multiple words that support an interpretation, separate them with a semicolon.\n
Here are two examples of the cases you will encounter:
1. An example with heterographic puns (words that sound similar) - 
The magician got so mad that he pulled his hare out.
{{
  "pun_word": "hare",
  "target_word": "hair",
  "source_sense": "magician",
  "target_sense": "angry;pulled"
}}
2. An example with homographic puns (same words with different senses) -
I used to be a banker but I lost interest.
{{
  "pun_word": "interest",
  "target_word": "interest",
  "source_sense": "banker",
  "target_sense": "used;to;be"
}}
Identify the pun word in the input sentence and predict the target word. Also, pick the supporting words for each sense. Generate a response in the form of a valid JSON object with four keys: pun_word, target_word, source_sense and target_sense.\n
### INPUT:\n
'''

In [18]:
results = []
i = 0
for _, row in test.iterrows():
  prepared_prompt = prompt + row['text'] + "\n### OUTPUT:\n"
  json_out = ''
  attempt = 0
  while True:
    attempt += 1
    json_out = generate(prepared_prompt, attempt)
    parsedJson = parseResult(json_out, row)
    print(f"{i} -- {attempt}")
    if parsedJson is not None:
      parsedJson['text'] = row['text']
      results.append(parsedJson)
      i+=1
      break

Llama.generate: prefix-match hit


0 -- 1



llama_print_timings:        load time =    3598.42 ms
llama_print_timings:      sample time =       6.21 ms /    52 runs   (    0.12 ms per token,  8376.29 tokens per second)
llama_print_timings: prompt eval time =    1647.81 ms /    17 tokens (   96.93 ms per token,    10.32 tokens per second)
llama_print_timings:        eval time =    2614.43 ms /    51 runs   (   51.26 ms per token,    19.51 tokens per second)
llama_print_timings:       total time =    4354.17 ms
Llama.generate: prefix-match hit


1 -- 1



llama_print_timings:        load time =    3598.42 ms
llama_print_timings:      sample time =       6.71 ms /    60 runs   (    0.11 ms per token,  8941.88 tokens per second)
llama_print_timings: prompt eval time =     186.75 ms /    20 tokens (    9.34 ms per token,   107.10 tokens per second)
llama_print_timings:        eval time =    3028.77 ms /    59 runs   (   51.34 ms per token,    19.48 tokens per second)
llama_print_timings:       total time =    3308.68 ms
Llama.generate: prefix-match hit


...


llama_print_timings:        load time =    3598.42 ms
llama_print_timings:      sample time =       5.18 ms /    54 runs   (    0.10 ms per token, 10428.74 tokens per second)
llama_print_timings: prompt eval time =     189.77 ms /    22 tokens (    8.63 ms per token,   115.93 tokens per second)
llama_print_timings:        eval time =    2722.36 ms /    53 runs   (   51.37 ms per token,    19.47 tokens per second)
llama_print_timings:       total time =    2984.85 ms
Llama.generate: prefix-match hit


316 -- 1



llama_print_timings:        load time =    3598.42 ms
llama_print_timings:      sample time =       5.33 ms /    53 runs   (    0.10 ms per token,  9945.58 tokens per second)
llama_print_timings: prompt eval time =     199.12 ms /    30 tokens (    6.64 ms per token,   150.66 tokens per second)
llama_print_timings:        eval time =    2674.55 ms /    52 runs   (   51.43 ms per token,    19.44 tokens per second)
llama_print_timings:       total time =    2947.98 ms
Llama.generate: prefix-match hit


317 -- 1



llama_print_timings:        load time =    3598.42 ms
llama_print_timings:      sample time =       5.78 ms /    56 runs   (    0.10 ms per token,  9681.88 tokens per second)
llama_print_timings: prompt eval time =     195.32 ms /    28 tokens (    6.98 ms per token,   143.35 tokens per second)
llama_print_timings:        eval time =    2826.93 ms /    55 runs   (   51.40 ms per token,    19.46 tokens per second)
llama_print_timings:       total time =    3103.62 ms


In [19]:
results_df = pd.DataFrame(results, columns=['text', 'pun_word', 'target_word', 'source_sense', 'target_sense'])
results_df.to_json('outputs-task2-hetero-openocra7b-nohumor.json', orient="records")
len(results_df)

318

In [20]:
def cprf(true, pred):
  correct = (true['pun_word'].str.strip().str.lower() == pred['pun_word'].str.strip().str.lower()).sum()
  coverage = len(pred) / len(true)
  precision = correct / len(pred)
  recall = correct / len(true)
  f1 = (2 * precision * recall) / (precision + recall)
  return coverage, precision, recall, f1

In [21]:
c, p, r, f1 = cprf(test, results_df)
print(f"Coverage: {c:.4f}")
print(f"Precision: {p:.4f}")
print(f"Recall: {r:.4f}")
print(f"F1-Score: {f1:.4f}")

Coverage: 1.0000
Precision: 0.6730
Recall: 0.6730
F1-Score: 0.6730
